In [41]:
# 展示名、展示説明、展示キーワードから分類表をもとに分類する

import json

# 分類表の読み込み
with open('category.json', 'r', encoding="utf8") as f:
    classification = json.load(f)

category_numbers = [x["category_number"] for x in classification]
category_names = [x["category_name"] for x in classification]

In [42]:
import openai
import os
from pprint import pprint

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

system = "あなたは博物館のキュレーターです。"
category_input = "以下は分類表です。以下の質問についてこの表を使用して答えてください。\n" + "\n".join([f"{num} {name}" for num, name in zip(category_numbers, category_names)])
response_format = """回答は以下の形式で行ってください。
[{
    "category_number": {category_number},
    "category_name": {category_name},
    "category_reason": {そのカテゴリを行った理由}
},
~~~
~~~
~~~
{
    "category_number": {category_number},
    "category_name": {category_name},
    "category_reason": {そのカテゴリを行った理由}
}]
"""


question = """以下与えられたJSONの展示について与えられた分類表をもとに分類してください。
{
    "exhibition_name": "開館30周年記念　特別展 立原位貫　〜想像力から創造力へ〜",
    "exhibition_description": "四日市ゆかりの木版画家・立原位貫（1951-2015）は、独学で江戸期浮世絵の魅力を現代に蘇らせ、さらにはオリジナルでも粋で個性的な作品を創り出しました。大英博物館に収蔵されるなど、世界に誇れる画家の足跡を辿り、その面白さ、美しさを紹介します。",
    "exhibition_keywords": None
}
"""

messages = [
    {"role": "system", "content": system},
    {"role": "system", "content": category_input},
    {"role": "system", "content": response_format},
    {"role": "user", "content": question}
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages
)
result = response.choices[0]["message"]["content"].strip()
result_json = json.loads(result)
pprint(result_json)

print("分類結果")
for category in result_json:
    print(f"{category['category_number']} {category['category_name']}")

[{'category_name': '書. 書道',
  'category_number': '728',
  'category_reason': '展示のタイトルに「想像力から創造力へ」という表現があり、書や書道は美的な表現形式として想像力を創造力に昇華させる要素が含まれていることから、このカテゴリに分類しました。また、立原位貫が個性的な作品を創り出したという点も、書や書道の表現の個性を引き立てる要素となります。'}]
